In [ ]:
#preparing data_set by removing urls, @ and RTs from the tweet text:
import numpy as np
import re
import warnings

#Visualisation
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from IPython.display import display
from mpl_toolkits.basemap import Basemap
from wordcloud import WordCloud, STOPWORDS

#nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize

#matplotlib.style.use('ggplot')
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

In [ ]:
#Removing URLS and @ and twitter handles 
data_set['text'] = data_set['text'].apply(lambda x: re.sub(r"http\S+", "", x))    
data_set['text'] = data_set['text'].apply(lambda x: re.sub(r"@\S+", "", x))      
#Delete certain punctuations, put the text in lower case and delete the double space with the function apply.
data_set['text'] = data_set['text'].apply(lambda x: re.sub('[!@#$:).;,?&]', '', x.lower()))
data_set['text'] = data_set['text'].apply(lambda x: re.sub('  ', ' ', x))
#removing the letter rt that don't have anything in front or after them
data_set['text'] = data_set['text'].apply(lambda x: re.sub(r'(?<![a-z])rt(?![a-z])', '', x, flags=re.I))

In [ ]:
#sentiment analysis
import nltk
#nltk.download('wordnet')
#nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import TfidfVectorizer
#Sentiment analysis
data_set['text_lem'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in data_set['text']]       
#Here I utilized both TFIDF and removed stop words. Note that I did not do that for the machine learning part.
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000, min_df=10, stop_words='english', use_idf=True)
X = vectorizer.fit_transform(data_set['text_lem'].str.upper())
sid = SentimentIntensityAnalyzer()
data_set['sentiment_compound_polarity']=data_set.text_lem.apply(lambda x:sid.polarity_scores(x)['compound'])
data_set['sentiment_neutral']=data_set.text_lem.apply(lambda x:sid.polarity_scores(x)['neu'])
data_set['sentiment_negative']=data_set.text_lem.apply(lambda x:sid.polarity_scores(x)['neg'])
data_set['sentiment_pos']=data_set.text_lem.apply(lambda x:sid.polarity_scores(x)['pos'])
data_set['sentiment_type']=''
data_set.loc[data_set.sentiment_compound_polarity>0,'sentiment_type']='POSITIVE'
data_set.loc[data_set.sentiment_compound_polarity==0,'sentiment_type']='NEUTRAL'
data_set.loc[data_set.sentiment_compound_polarity<0,'sentiment_type']='NEGATIVE'


In [ ]:
#sentiment analysis graph for depressed tweets only
#subsetting by throwing out not-depressed
data_set_sentiment_depressed = data_set.loc[:,['depressed','sentiment_neutral', 'sentiment_type', 'text', 'text_lem']]
data_set_sentiment_depressed = data_set_sentiment_depressed[data_set_sentiment_depressed.depressed != 0]

data_set_sentiment = data_set_sentiment_depressed.groupby(['sentiment_type'])['sentiment_neutral'].count()
data_set_sentiment.rename("",inplace=True)
explode = (1, 0, 0)
plt.subplot(221)
data_set_sentiment.transpose().plot(kind='barh',figsize=(20, 20))
plt.title('Sentiment of Tweets by Depressed People', bbox={'facecolor':'0.8', 'pad':0})
plt.subplot(222)
data_set_sentiment.plot(kind='pie',figsize=(20, 20),autopct='%1.1f%%',shadow=True,explode=explode)
plt.legend(bbox_to_anchor=(1, 1), loc=3, borderaxespad=0.)
plt.title('Sentiment of Tweets by Depressed People', bbox={'facecolor':'0.8', 'pad':0})
plt.show()


In [ ]:
#sentiment analysis graph for not-depressed tweets only
#subsetting by throwing out depressed
data_set_sentiment_not_depressed = data_set.loc[:,['depressed','sentiment_neutral', 'sentiment_type', 'text', 'text_lem']]
data_set_sentiment_not_depressed = data_set_sentiment_not_depressed[data_set_sentiment_not_depressed.depressed != 1]

data_set_sentiment = data_set_sentiment_not_depressed.groupby(['sentiment_type'])['sentiment_neutral'].count()
data_set_sentiment.rename("",inplace=True)
explode = (1, 0, 0)
plt.subplot(221)
data_set_sentiment.transpose().plot(kind='barh',figsize=(20, 20))
plt.title('Sentiment of Tweets by Not Depressed People', bbox={'facecolor':'0.8', 'pad':0})
plt.subplot(222)
data_set_sentiment.plot(kind='pie',figsize=(20, 20),autopct='%1.1f%%',shadow=True,explode=explode)
plt.legend(bbox_to_anchor=(1, 1), loc=3, borderaxespad=0.)
plt.title('Sentiment of Tweets by Not Depressed People', bbox={'facecolor':'0.8', 'pad':0})
plt.show()


In [ ]:
#Word Cloud depressed Content Words
#Deleting the punctuations, the URLs, putting the test in a lower case, extractting the username for examples
def wordcloud(tweets,col):
    stopwords = set(STOPWORDS)
    wordcloud = WordCloud(background_color = "white", stopwords = stopwords, random_state = 2016).generate(" ".join([i for i in tweets[col]]))
    plt.figure( figsize = (10,5), facecolor = 'k')
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title("")
wordcloud(data_set_sentiment_depressed, 'text_lem')

In [ ]:
#Word Cloud not depressed Content Words
#Deleting the punctuations, the URLs, putting the test in a lower case, extractting the username for examples
def wordcloud(tweets,col):
    stopwords = set(STOPWORDS)
    wordcloud = WordCloud(background_color = "white", stopwords = stopwords, random_state = 2016).generate(" ".join([i for i in tweets[col]]))
    plt.figure( figsize = (10,5), facecolor = 'k')
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title("Good Morning Datascience+")
wordcloud(data_set_sentiment_not_depressed, 'text_lem')

In [ ]:
#Adding to dataframe how many characters in tweet
#did not end up using it, but here it is
#data_set['text_len_char'] = data_set['text'].apply(len)

#Based on previous reserach I hypothesized that depressed tweets would have a higher percentage of "i" relative to other words
#Adding to dataframe percentage of 'i' in tweet
#finding how many words in each tweet
data_set['text_len_word'] = data_set['text'].apply(lambda x: re.findall(r'(\S+)', x))
data_set['text_len_word'] = data_set['text_len_word'].apply(len)
#finding how many times "i" appears in each tweet by only keeping the free standing "i"s and then counting them
data_set['text_i'] = data_set['text_lem'].apply(lambda x: re.findall(r'(?:\s|^)i(?=\s|$)', x))
data_set['text_i'] = data_set['text_i'].apply(len)
#Finding what percentage of "i" is of all tweet words
data_set['text_i_percent'] = (data_set['text_i']/data_set['text_len_word'])*100

#Examining the difference between depressed and non-depressed "i" usage:
data_set.groupby(['depressed'])['text_i_percent'].mean()
#Graphing it:
import altair as alt

# for the notebook only (not for JupyterLab) run this command once per session
alt.renderers.enable('notebook')
alt.Chart(data_set).mark_bar().encode(
    x='depressed',
    y='average(text_i_percent)'
)

In [ ]:
#Histogram
import matplotlib.pyplot as plt
plt.figure()
data_set['text_i_percent'].plot.hist()

In [ ]:
#If you saved the timestamp data you can also examine whether the time of day depressed vs not-depressed tweet differs:
data_set['created_at1'] = pd.to_datetime(data_set['created_at'])
#keeping only the hour
data_set['created_at1'] = data_set['created_at1'].dt.hour
data_set['late_night'] = data_set['created_at1'].apply(lambda x: 'late night' if 8>x or x>21 else 'not late night')

In [ ]:
#To save data as a pickle file:
import jsonpickle
import pandas as pd
data_set = pd.read_pickle('/home/twitterdata.pickle')

In [ ]:
#To save it as a CSV:
data_set.to_csv('/home/twitterdata.csv', sep = ',')